In [ ]:
from pybeataml.load_data_from_synpase import syn

In [ ]:
import networkx as nx
import numpy as np
import seaborn as sns
import pandas as pd
import igraph
import matplotlib.pyplot as plt

In [ ]:
from magine.networks.visualization import draw_igraph, draw_graphviz, draw_mpl
from magine.networks.visualization.notebooks import view

In [ ]:
syn.get('syn26997954')

In [ ]:
def get_networks_from_folder(folder):
    nodes, networks = [], []
    for i in syn.getChildren(folder):
        fname = syn.get(i['id']).path
        if fname.endswith('.gml'):
    #         net = nx.read_gml(syn.get(i['id']).path, label = 'name')
            net = igraph.load(syn.get(i['id']).path,)
            net.name = syn.get(i['id']).files[0]
            nodes.append(set(net.vs['name']))
            networks.append(net)
    return nodes, networks
nodes, gmls = get_networks_from_folder('syn26997954')

In [ ]:
[gmls[i].name for i in range(len(nodes))]
k5_nodes = [nodes[i] for i in range(5)]
k8_nodes = [nodes[i] for i in range(5, len(nodes))]
print(len(k5_nodes))
print(len(k8_nodes))

In [ ]:

def load_dist_matrix(nodes):
    node_dist = np.zeros((len(nodes), len(nodes)))
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            node_dist[i,j] = len(nodes[i].intersection(nodes[j]))
    return node_dist
# node_dist += node_dist.T
k5_node_dist = load_dist_matrix(k5_nodes)
k8_node_dist = load_dist_matrix(k8_nodes)

In [ ]:
plt.figure(figsize=(6,6))
plt.title("K=8 graph node intersections")
sns.heatmap(k5_node_dist, square=True, annot=True, fmt=".0f", linewidths=0.01);
plt.figure(figsize=(6,6))
plt.title("K=8 graph node intersections")
sns.heatmap(k8_node_dist, square=True, annot=True, fmt=".0f", linewidths=0.01);

In [ ]:
gmls[1].name

#### Simple plots of networks, not really useful but here for now

In [ ]:
layout = gmls[1].layout_reingold_tilford_circular()
gmls[1].vs["label"] = gmls[1].vs["name"]
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
igraph.plot(gmls[1], layout=layout, target=ax)

In [ ]:
igraph.plot(gmls[1], layout=layout)

Run enrichment using EnrichR.

There are multiple options for gene_set_lib.

In [ ]:
from magine.enrichment.enrichr import Enrichr, get_libraries
e = Enrichr()

In [ ]:
sorted(get_libraries())


In [ ]:
results = e.run_samples(nodes, [i.name for i in gmls], gene_set_lib='Reactome_2016')

Returns an EnrichmentResult from magine. Has a lot of capabilities (https://magine.readthedocs.io/en/latest/)

In [ ]:
results.head()

In [ ]:
# cleanup names
results.term_name = results.term_name.str.split('_hsa').str.get(0)

In [ ]:
k5 = [f'k5_cluster{i}.gml' for i in range(1,6)]
k5_enrichment = results.filter_multi(sample_id=k5)
k5_enrichment = k5_enrichment.loc[k5_enrichment.n_genes>5]

k8 = [f'k8_cluster{i}.gml' for i in range(1,9)]
k8_enrichment = results.filter_multi(sample_id=k8)
k8_enrichment = k8_enrichment.loc[k8_enrichment.n_genes>5]

In [ ]:
k5_enrichment.describe()

In [ ]:
# get counts per term, goal is to find unique terms per network
counts = k8_enrichment.pivoter(index='sample_id', columns='term_name', values='combined_score').count().sort_values()

k8_only_1 = k8_enrichment.loc[k8_enrichment.term_name.isin(counts[counts==1].index.values)].copy()

k8_only_1.sig.remove_redundant(threshold=.25, level='sample').dist_matrix();
k8_only_1.sig.remove_redundant(threshold=.25, level='sample', sort_by='n_genes').heatmap(
    convert_to_log=True,
    cluster_row=False,
    figsize=(4, 12),
    y_tick_labels=True,
    linewidths=0.01
#     sort_row='max'
);

In [ ]:
# same as above but for k5
counts = k5_enrichment.pivoter(
    index='sample_id', 
    columns='term_name', 
    values='combined_score'
).count().sort_values()

k5_only_1 = k5_enrichment.loc[k5_enrichment.term_name.isin(counts[counts<6].index.values)].copy()
k5_only_1.sig.remove_redundant(threshold=.25, level='sample', ).heatmap(
    convert_to_log=True,
    cluster_row=False,
    figsize=(4, 12),
    y_tick_labels=True,
    linewidths=0.01
#     sort_row='max'
);

In [ ]:
k8_enrichment.remove_redundant(threshold=.5, level='dataframe').heatmap(
    convert_to_log=False,
    cluster_row=False,
    min_sig=4
);

In [ ]:
k8_enrichment.remove_redundant(threshold=.5, level='dataframe').heatmap(
    convert_to_log=True,
    cluster_row=True,
    figsize=(12,16),
    
);
plt.savefig("k8_network_expand.png", bbox_inches='tight', dpi=300)

In [ ]:
k5_enrichment.remove_redundant(threshold=.5, level='sample').heatmap(
    figsize=(12,16),
    convert_to_log=True,
    cluster_row=True,
    
);